# ZON Comments SQL Analysis

## Setup the DB

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import sqlalchemy as sqla
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import date
from IPython.display import set_matplotlib_formats
from plotting import get_plot_size

#set_matplotlib_formats('png', 'pdf')

# Graph settings
sns.set(context='paper',style="whitegrid", palette='muted')
#plt.rc("figure", figsize=(10,6)) # global figsize

# Enable Graph exporting to Latex
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",  # use serif/main font for text elements
    "pgf.rcfonts": False,     # don't setup fonts from rc parameters
    "pgf.preamble": """
        \\usepackage[utf8x]{inputenc}
        \\usepackage[T1]{fontenc}
        \\usepackage[detect-all]{siunitx}
    """
})

# Load DB URI from .env file
load_dotenv()
DB_URI = os.getenv("DB_URI")

engine = sqla.create_engine(DB_URI)
CONNECTION = engine.connect()

TABLE = 'zon_comments_cleaned'

# Foundational Analysis

#### Available columns

In [ ]:
pd.read_sql(f'SELECT * FROM {TABLE} LIMIT 1', CONNECTION).columns

#### Total # of comments

In [ ]:
total_no_comments = pd.read_sql(f'SELECT count(comment_id) FROM {TABLE}', CONNECTION)
total_no_comments

### Total # of users

In [ ]:
pd.read_sql(f'SELECT COUNT(DISTINCT user_id) FROM {TABLE}', CONNECTION)

#### Total # of articles

In [ ]:
pd.read_sql(f'SELECT COUNT(DISTINCT article_id) FROM {TABLE}', CONNECTION)

#### Time range (first and last comment)

In [ ]:
pd.read_sql(f'SELECT min(created), max(created) FROM {TABLE}', CONNECTION)

Show the first and last comment

In [ ]:
pd.read_sql(f"""
SELECT *
FROM {TABLE}
WHERE created =
        (SELECT min(created)
         FROM {TABLE})
UNION ALL
SELECT *
FROM {TABLE}
WHERE created =
        (SELECT max(created)
         FROM {TABLE})
""", CONNECTION)

### Comment statistics

#### Average comment length

In [ ]:
pd.read_sql(f'SELECT avg(char_length(raw_content)) from {TABLE}', CONNECTION)

In [ ]:
pd.read_sql(f"""
SELECT percentile_disc(0.5)
WITHIN GROUP (ORDER BY char_length({TABLE}.raw_content))
FROM {TABLE}""", CONNECTION)

#### Average number of words per comment

In [ ]:
pd.read_sql(f"""
SELECT avg(array_length(regexp_split_to_array(raw_content, '\s'), 1))
FROM {TABLE}""", CONNECTION)

#### Median number of words per comment

In [ ]:
pd.read_sql(f"""
SELECT percentile_disc(0.5) within
GROUP (ORDER BY array_length(regexp_split_to_array({TABLE}.raw_content, '\s'), 1))
FROM {TABLE};
""", CONNECTION)

#### Top 10 articles with most comments

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(pd.read_sql(f"""
    SELECT article_id, count(comment_id)
    FROM {TABLE}
    WHERE parent_id IS NULL
    GROUP BY article_id
    ORDER BY count DESC
    LIMIT 10
    """, CONNECTION))

#### Number of Comments with no answer

In [ ]:
unanswered = pd.read_sql(f"""
SELECT count(*) FROM {TABLE} AS parents
LEFT JOIN {TABLE} AS answers ON parents.comment_id = answers.parent_id
WHERE answers.parent_id IS NULL;
""", CONNECTION)
unanswered

In [ ]:
unanswered / total_no_comments

#### Number of Top-Level Comments with no answers 

In [ ]:
unanswered_top_level = pd.read_sql(f"""
SELECT count(*) FROM {TABLE} AS parents
LEFT JOIN {TABLE} AS answers ON parents.comment_id = answers.parent_id
WHERE answers.parent_id IS NULL
AND parents.parent_id IS NULL;
""", CONNECTION)

In [ ]:
unanswered_top_level / total_no_comments

In [ ]:
1 - unanswered_top_level / total_no_comments

#### Published but not visible comments
Comments where the published field is `true` but visible is `false`.

In [ ]:
published = pd.read_sql(f"""
SELECT count(*) FROM {TABLE} WHERE published=true AND visible=false
""", CONNECTION)
published

#### Published and visible comments
Comments where the published field is `true` and visible is `true`.

In [ ]:
visible = pd.read_sql(f"""
SELECT count(*) FROM {TABLE} WHERE published=true AND visible=true
""", CONNECTION)
visible

Percent of published and visible comments of all comments

In [ ]:
(visible / total_no_comments) * 100

In [ ]:
comments_per_timespan = pd.read_sql(f"""
SELECT
    date_trunc('day', created)::date date,
    count(*) total,
    sum(CASE WHEN published THEN 1 ELSE 0 END) published,
    sum(CASE WHEN visible THEN 1 ELSE 0 END) visible
FROM {TABLE} t
GROUP BY date
ORDER BY date ASC
""", CONNECTION)

comments_per_timespan

In [ ]:
comments_per_timespan.describe()

In [ ]:
import matplotlib.dates as mdates

# add date ordinal because matplotlib cannot deal with dates
comments_per_timespan['date_ordinal'] = comments_per_timespan['date'].map(lambda date: date.toordinal())

ax = sns.regplot(
    data=comments_per_timespan,
    x='date_ordinal',
    y='total',
    ci=None,
    marker='+',
    scatter_kws={"color": "silver"}
)
ax.set_xlim(comments_per_timespan['date_ordinal'].min() - 1, comments_per_timespan['date_ordinal'].max() + 1)
ax.set_ylim(4000, 18000)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Comments')
x_ticks = comments_per_timespan.iloc[::(len(comments_per_timespan)//6), :]['date_ordinal']
ax.set_xticks(x_ticks)
labels = [date.fromordinal(int(item)).strftime("%y-%m-%d") for item in ax.get_xticks()]
ax.set_xticklabels(labels, ha='left')
ax.set_yticks(list(range(4000,18001,2000)))
ax.set_yticklabels(list(range(4000,18001,2000)));

# Save plot with siunitx number formatting
#ax.set_yticklabels(map(lambda l: f'\\num{{{l}}}', list(range(4000,18001,2000))))
#plt.savefig('figures/daily-comments-regression.pgf')

### Departments (Ressorts)

#### Comments per Department

Using the original department column (no `department_mapping` join)

In [ ]:
with pd.option_context('display.max_rows', None):
    display(pd.read_sql(f"""
        SELECT department,
               count(department)
        FROM {TABLE}
        GROUP BY department
        ORDER BY count DESC""", CONNECTION))

With `department_mapping` join:

In [ ]:
departments = pd.read_sql(f'''
    SELECT
        b.target as department,
        count(a.department) comment_count
    FROM {TABLE} a
    INNER JOIN department_mapping b
    ON a.department = b.source
    GROUP BY b.target
    ORDER BY comment_count asc
''', CONNECTION)

In [ ]:
fig, ax = plt.subplots()
bars = ax.barh(y=departments.department, width=departments.comment_count, height=0.75)
ax.set_xscale("log")
ax.set_xlim(1000, 10000000)
ax.set_xlabel("Number of Comments", labelpad=12)
ax.set_ylabel("Department")
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top') 
plt.minorticks_off()
#ax.get_xaxis().set_ticks([])
#formatter = ScalarFormatter()
#formatter.set_scientific(False)
#ax.xaxis.set_major_formatter(formatter)
#ax.set_xticks([0, 1000, 10000, 100000])

# add numbers to bars
import locale
locale.setlocale(locale.LC_ALL, '')
for bar in bars:
    width = bar.get_width()
    text = f'{width:,}'
    label_y_pos = bar.get_y() + bar.get_height() / 2
    ax.text(
        width * 1.05,
        label_y_pos,
        s=text,
        va='center',
        ha='left',
        color='dimgray',
        size=10,
        fontfamily='sans-serif'
    )

fig.tight_layout()
# plt.savefig('figures/department_comment_barchart.pgf')

In [ ]:
#sns.barplot(x="comment_count", y="department", data=departments)

## User Stats

In [ ]:
users = pd.read_sql('SELECT * FROM user_stats', CONNECTION)
users.describe()

In [ ]:
ax = sns.scatterplot(data=users, x="count_total", y="count_replies_to_others")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylim((1, 100000))
ax.set_xlim((1, 100000))

#ax.set_yticklabels(map(lambda l: f'\\num{{{l}}}', ax.get_yticks()))

In [ ]:
ax = sns.scatterplot(data=users, x="count_not_visible", y="count_top_level")



In [ ]:
ax = sns.scatterplot(data=users, x="avg_comment_length", y="count_total")
#ax.set_yscale("log")
#ax.set_ylim((1, 100000))

## Answerer Graph Statistics

In [ ]:
answ_graph = pd.read_sql(f"""
    SELECT *
    FROM answerer_graph
    """, CONNECTION)

In [ ]:
answ_graph.describe()

In [ ]:
quantiles = np.arange(0.0,1.000001,0.01)
percentiles = answ_graph.quantile(q=quantiles).drop(columns=['poster', 'answerer'])

percentiles

In [ ]:
ax = sns.scatterplot(data=percentiles)
ax.set_xticks(np.arange(0.0,1.5,0.5))
ax.set_yticks([0, 1, 2,3,4, 5, 10, 50, 100, 1000, 2500])
ax.set_xscale("log")
ax.set_yscale("log")